<h2>Read Content From PDf - To be Used For RAG</h2>
Post Reading, store in a file as Json

In [9]:
from PyPDF2 import PdfReader

reader = PdfReader("sourceData\A Brief History of Time .. Stephen Hawking.pdf")
number_of_pages = len(reader.pages)
bookPages = ""

#Read content from all pages, compile them into One Txt File
for page in range(number_of_pages):
    page = reader.pages[page]
    text = page.extract_text()
    bookPages = bookPages + text


In [13]:
processedString = bookPages.replace("\n", "")
print(processedString)

Chapter 1  - Our Picture of the UniverseChapter 2  - Space and TimeChapter 3  - The Expanding UniverseChapter 4  - The Uncertainty PrincipleChapter 5  - Elementary Particles and the Forces of NatureChapter 6  - Black HolesChapter 7  - Black Holes Ain't So BlackChapter 8  - The Origin and Fate of the UniverseChapter 9  - The Arrow of TimeChapter 10  - Wormholes and Time TravelChapter 11  - The Unification of PhysicsChapter 12  - ConclusionGlossaryAcknowledgments & About The Author  FOREWARDI didn’t write a foreword to the original edition of A Brief History of Time. That was done by Carl Sagan. Instead,I wrote a short piece titled “Acknowledgments” in which I was advised to thank everyone. Some of thefoundations that had given me support weren’t too pleased to have been mentioned, however, because it led toa great increase in applications.I don’t think anyone, my publishers, my agent, or myself, expected the book to do anything like as well as it did.It was in the London Sunday Times be

In [14]:
import json
content = {"text": bookPages}

with open('sourceData\content.json', 'w') as file:
    json.dump(content, file)